<a href="https://colab.research.google.com/github/Nithinan-Srikongphan/python_dataviz2024/blob/main/(Test02)Find_Murder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ให้สืบหาฆาตรกร ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง SQL City โดยเริ่มจากการตรวจสอบ ข้อมูลคดี (crime_scene_report.csv)

In [43]:
import pandas as pd
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

df_crime = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/crime_scene_report.csv')
df_person = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/person.csv')
df_interview = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/interview.csv')
df_income = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/income.csv')
df_getfitnow_member = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/get_fit_now_member.csv')
df_getfitnow_checkin = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/get_fit_now_check_in.csv')
df_facebook_checkin = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/facebook_event_checkin.csv')
df_drivers_license = pd.read_csv('/content/drive/My Drive/data_viz_2024_/Exam_2/drivers_license.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
df_crime.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


In [45]:
# prompt: คดี murder เกิดขึ้นใน date 20180115 ที่เมือง SQL City

df_crime[(df_crime['date']==20180115) & (df_crime['city']=='SQL City')  & (df_crime['type']=='murder')]

,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


#ภาพจากกล้องวงจรปิดพบว่ามีพยาน 2 คน พยานคนแรกอาศัยอยู่ที่บ้านหลังสุดท้ายบน "Northwestern Dr" พยานคนที่สองชื่อแอนนาเบล อาศัยอยู่ที่ไหนสักแห่งบนถนนแฟรงคลิน

In [46]:
df_person.head()

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


In [47]:
# prompt: address_street_name is Northwestern Dr and that home has last address number

df_person[df_person["address_street_name"] == "Northwestern Dr"].sort_values("address_number", ascending=False).iloc[0]


,499
id,14887
name,Morty Schapiro
license_id,118009
address_number,4919
address_street_name,Northwestern Dr
ssn,111564949


In [48]:
#แยกชื่อนามสกุล

In [49]:
# prompt: แยกชื่อและนามสกุล

df_person[['first_name','last_name']] = df_person['name'].str.split(' ',expand=True)
df_person.head()


,id,name,license_id,address_number,address_street_name,ssn,first_name,last_name
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076,Christoper,Peteuil
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044,Kourtney,Calderwood
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512,Muoi,Cary
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061,Era,Moselle
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684,Trena,Hornby


In [60]:
# prompt: first_name is Annabel in df_person

df_person[df_person['first_name']=='Annabel']


,id,name,license_id,address_number,address_street_name,ssn,first_name,last_name
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143,Annabel,Miller


In [61]:
df_interview.head()

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n


In [52]:
# prompt: person_id is 14887 and 16371

df_interview[(df_interview['person_id']==14887) | (df_interview['person_id']==16371)]


,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


#ฉันได้ยินเสียงปืนก็เห็นชายคนหนึ่งวิ่งออกไป เขามีกระเป๋า "Get Fit Now Gym" หมายเลขสมาชิกบนกระเป๋าขึ้นต้นด้วย "48Z" เฉพาะสมาชิกระดับโกลด์เท่านั้นที่มีถุงเหล่านั้น ชายคนนั้นขึ้นรถพร้อมจานที่มี "H42W"

#ฉันเห็นการฆาตกรรมเกิดขึ้น และฉันก็จำฆาตกรได้จากยิมของฉันตอนที่ฉันกำลังออกกำลังกายเมื่อสัปดาห์ที่แล้วในวันที่ 9 มกราคม

In [53]:
df_getfitnow_member.head()

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver


In [54]:
# prompt: membership_status is gold id start "48Z"

df_getfitnow_member[df_getfitnow_member['membership_status']=='gold'] [df_getfitnow_member['id'].str.startswith('48Z')]


<ipython-input-54-5fd5564c71d6>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_getfitnow_member[df_getfitnow_member['membership_status']=='gold'] [df_getfitnow_member['id'].str.startswith('48Z')]


,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [55]:
df_getfitnow_checkin.head()

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019


In [56]:
df_getfitnow_checkin[(df_getfitnow_checkin['check_in_date'] == 20180109) &
                     ((df_getfitnow_checkin['membership_id'] == '48Z55') |
                      (df_getfitnow_checkin['membership_id'] == '48Z7A'))]

,membership_id,check_in_date,check_in_time,check_out_time
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700


In [57]:
df_drivers_license.head()

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS


In [58]:
# prompt: รถที่มีป้ายทะเบียนที่ประกอบด้วยตัวอักษร H,4,2, W และผู้ชายขับ

# หาป้ายทะเบียนที่มี H,4,2, W
license_plates = df_drivers_license[df_drivers_license['plate_number'].str.contains('H') &
                                     df_drivers_license['plate_number'].str.contains('4') &
                                     df_drivers_license['plate_number'].str.contains('2') &
                                     df_drivers_license['plate_number'].str.contains('W')]
#เลือกเฉพาะผู้ชาย
license_plates = license_plates[license_plates['gender'].isin(['male'])]
license_plates


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [59]:
df_person[df_person['license_id'].isin(license_plates['id'])]

,id,name,license_id,address_number,address_street_name,ssn,first_name,last_name
4664,51739,Tushar Chandra,664760,312,Phi St,137882671,Tushar,Chandra
6327,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,Jeremy,Bowers


#ขอคาดการณ์ว่า Jeremy Bowers เป็นฆาตกร เนื่องจากมีรายชื่อตรงกับคนที่เข้าไปใช้ยิมพอดี